# DT Assignment

# Data Loading

In [3]:
import pandas as pd 
import numpy as np

In [6]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [11]:
pd_data.iloc[:,-1].value_counts().values

array([9, 5], dtype=int64)

In [76]:
def get_gini(df, label):
    total = len(df)
    gini = 0
    for v in df[label].value_counts().values:
        gini += np.power((v/total), 2)
        
    return 1-gini

In [77]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [78]:
from itertools import combinations

def get_binary_split(df, attribute):
    result = []
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    for i in range(1, len(uniques)):
        for c in combinations(uniques, i):
            result.append(list(c))
    return result

In [40]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [81]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    for split in binary_split:
        key = '_'.join(split)
        pos = df[df[attribute].apply(lambda x: x in split)]
        neg = df[df[attribute].apply(lambda x: x not in split)]
        value = get_gini(pos, label) * len(pos)/n + get_gini(neg, label) * len(neg)/n
        result[key] = value
    return result

In [82]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.44285714285714284,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.44285714285714284}

In [83]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.44285714285714284,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.44285714285714284}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [84]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.44285714285714284)

In [85]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [91]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    min_value = min(get_attribute_gini_index(pd_data, feature, 'class_buys_computer').items())[1]
    print(f'Minimum Gini index of {feature} : {min_value:.4f}')

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [92]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507936,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youth_middle_aged': 0.45714285714285713,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507936}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [155]:
from math import log2

def getEntropy(df, feature) :
    total = len(df)
    entropy = 0
    for v in df[feature].value_counts().values:
        p = v / total
        entropy += (p * log2(p))
        
    return -entropy    

In [156]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [260]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    result = {}
    base_entropy = getEntropy(df, feature)
    columns = list(df.columns)
    columns.remove(feature)
    n = len(df)

    for column in columns:
        entropy = 0
        ff = df[column].unique()
        for f in ff:
            temp = df[df[column]==f]
            entropy  += getEntropy(temp, feature) * len(temp) / n
        print(f'{column} : {base_entropy - entropy}')

In [261]:
getGainA(pd_data, "class_buys_computer")

age : 0.24674981977443922
income : 0.02922256565895487
student : 0.15183550136234159
credit_rating : 0.04812703040826949
